In [1]:
#| default_exp auth

# User authentication and session management

In [2]:
#| hide
from nbdev.showdoc import *


In [3]:
#| export
from typing import Optional, Dict, Tuple
import time
import requests
from requests.exceptions import RequestException
from datetime import datetime
import os
from pathlib import Path
import logging
from wardbuddy.utils import setup_logger

logger = setup_logger(__name__)

In [4]:
#| export
class AuthManager:
    """Manages user authentication and session tracking."""
    
    def __init__(self, api_url: Optional[str] = None):
        """Initialize auth manager with fault tolerance."""
        try:
            self.api_url = api_url or os.getenv("API_URL")
            
            if not self.api_url:
                logger.warning("No API_URL provided, running in limited mode")
                self.api_url = "http://localhost:8000"  # Default fallback
            
            self.token: Optional[str] = None
            self.session_id: Optional[int] = None
            
            logger.info(f"Auth manager initialized with API: {self.api_url}")
            
        except Exception as e:
            logger.error(f"Auth manager initialization failed: {str(e)}")
            # Don't raise the exception, allow degraded operation
            self.api_url = "http://localhost:8000"
            self.token = None
            self.session_id = None

    def check_server_connection(self) -> bool:
        """Check if the server is accessible with detailed error logging."""
        try:
            api_url = self.api_url.rstrip('/')
            logger.info(f"Attempting to connect to {api_url}/health")
            
            # Add headers for CORS
            headers = {
                'Origin': 'https://huggingface.co',
                'Access-Control-Request-Method': 'GET',
            }
            
            response = requests.get(
                f"{api_url}/health", 
                timeout=5,
                headers=headers,
                verify=False  # Only for testing
            )
            
            logger.info(f"Server response status: {response.status_code}")
            logger.info(f"Server response: {response.text}")
            logger.info(f"Response headers: {dict(response.headers)}")
            
            return response.status_code == 200
            
        except requests.exceptions.ConnectionError as e:
            logger.error(f"Connection error: {str(e)}")
            return False
        except requests.exceptions.Timeout as e:
            logger.error(f"Timeout error: {str(e)}")
            return False
        except Exception as e:
            logger.error(f"Unexpected error checking connection: {str(e)}")
            return False
    
    def login(self, email: str, password: str) -> Tuple[bool, str]:
        """Log in user and get access token."""
        # Add connection check here
        if not self.check_server_connection():
            logger.error(f"Could not connect to server at {self.api_url}")
            raise ConnectionError(f"Could not establish connection to {self.api_url}")

        try:
            response = requests.post(
                f"{self.api_url}/token",
                data={"username": email, "password": password}
            )
            
            if response.status_code == 200:
                self.token = response.json()["access_token"]
                return True, ""
            return False, response.json().get("detail", "Login failed")
            
        except Exception as e:
            logger.error(f"Login error: {str(e)}")
            return False, f"Error during login: {str(e)}"
    
    def register(self, email: str, password: str) -> Tuple[bool, str]:
        """Register new user with improved error handling."""
        # Add connection check here
        if not self.check_server_connection():
            logger.error(f"Could not connect to server at {self.api_url}")
            # Don't raise error, return failure instead
            return False, f"Could not connect to server. Please try again later."

        try:
            # Ensure API URL is properly formatted
            api_url = self.api_url.rstrip('/')
            
            headers = {
                'Origin': 'https://huggingface.co',
                'Content-Type': 'application/json'
            }
            
            response = requests.post(
                f"{api_url}/register",
                json={"email": email, "password": password},
                headers=headers,
                timeout=10
            )
            
            logger.info(f"Register response status: {response.status_code}")
            logger.info(f"Register response: {response.text}")
            
            # Try to parse response as JSON
            try:
                response_data = response.json()
            except requests.exceptions.JSONDecodeError:
                return False, f"Invalid response from server: {response.text}"
                
            if response.status_code == 200:
                return True, "Registration successful"
            elif response.status_code == 400:
                return False, response_data.get("detail", "Registration failed - invalid input")
            elif response.status_code == 409:
                return False, "User already exists"
            else:
                return False, f"Registration failed - server error ({response.status_code})"
                
        except requests.exceptions.ConnectionError:
            return False, "Could not connect to server"
        except requests.exceptions.Timeout:
            return False, "Server request timed out"
        except Exception as e:
            logger.error(f"Registration error: {str(e)}")
            return False, f"Unexpected error during registration: {str(e)}"
            
    def start_session(self, specialty: str, setting: str) -> Optional[int]:
        """Start new learning session."""
        if not self.token:
            return None
            
        try:
            response = requests.post(
                f"{self.api_url}/sessions",
                json={"specialty": specialty, "setting": setting},
                headers={"Authorization": f"Bearer {self.token}"}
            )
            
            if response.status_code == 200:
                self.session_id = response.json()["session_id"]
                return self.session_id
                
        except Exception as e:
            logger.error(f"Session start error: {str(e)}")
        return None
    
    def log_interaction(self, interaction_type: str, content: Optional[str] = None) -> bool:
        """Log user interaction."""
        if not self.token or not self.session_id:
            return False
            
        try:
            response = requests.post(
                f"{self.api_url}/interactions",
                params={"session_id": self.session_id},
                json={
                    "interaction_type": interaction_type,
                    "content": content
                },
                headers={"Authorization": f"Bearer {self.token}"}
            )
            
            return response.status_code == 200
            
        except Exception as e:
            logger.error(f"Interaction logging error: {str(e)}")
            return False

# Tests

In [7]:
#| hide
def test_auth_manager():
    """Test auth manager functionality."""
    # Skip if no API URL
    if not os.getenv("API_URL"):
        print("Skipping tests: No API_URL set")
        return
        
    auth = AuthManager()
    
    # Test registration
    success, msg = auth.register("test@example.com", "testpass123")
    assert success or "already exists" in msg.lower()
    
    # Test login
    success, msg = auth.login("test@example.com", "testpass123")
    assert success
    assert auth.token is not None
    
    # Test session
    session_id = auth.start_session("Internal Medicine", "Wards")
    assert session_id is not None
    
    # Test interaction logging
    success = auth.log_interaction("test_interaction", "test content")
    assert success
    
    print("Auth manager tests passed!")

if __name__ == "__main__":
    test_auth_manager()

Skipping tests: No API_URL set
